## Install PyTorch
````
# Linux / Binder
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio
````

### Import Torch

In [ ]:
import torch

### What is a Tensor?
A Tensor is a number, vector, matrix, or any n-dimensional array. A tensor has an additional property of uniformity in the data type of all entries and that they are always of a proper shape i.e., the lenghts of rows are all same, and the same goes with columns.

[Tensors: Reference](https://pytorch.org/docs/stable/torch.html)

In [ ]:
t0 = torch.tensor([[1,2,3],
                  [4,5,6],
                  [7,8,9]])
print(t0)
print(t0.dtype)
print(t0.shape)

In [ ]:
t1 = torch.tensor([[1,2,3],
                  [4,5,6],
                  [7.0,8,9]])
print(t1)
print(t1.dtype)
print(t1.shape)

Now, t1 is a 3x3 Tensor with dtype "float32". Since one entry (7.0) is a floating point entry, the rest will be converted to FP too.

Here is an example of a 3D Tensor of size 2x3x4. Similarly nD Tensors can also be constructed.

In [ ]:
t2 = torch.tensor([[[1,2,3,4],
                   [5,6,7,8],
                   [9,10,11,12]],
                   [[9,10,11,12],
                   [13,14,15,16],
                   [17,18,19,20]]])
print(t2)
print(t2.shape)

### Gradients
We can perform arithmetic operations on Torch tensors

In [ ]:
x = torch.tensor(2.,requires_grad=True)
w = torch.tensor(3.,requires_grad=True)
b = torch.tensor(7.,requires_grad=True)
y = w*x + b
print(y)

If you print **y**, you can see that it is a tensor with one entry = 13. We set ````requires_grad=True```` for making the tensors utilizable for certain functions. For example, we can perform Backprop as follows

In [ ]:
y.backward()
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

We obtain derivatives of y with respect to x,w and b i.e., dy/dx, dy/dw and dy/db as 3,2 and 1 respectively. If we had set ````requires_grad=False````, these would print out as ````None````. 

Default is ````requires_grad=False````

### Some Torch Functions

In [ ]:
t2 = torch.ones(3,4)
t3 = torch.full((3,4),7.384)
t4 = torch.cat((t2,t3))
t5 = t2.reshape(3,2,2)
print(f't2: {t2}\n')
print(f't3: {t3}\n')
print(f't4: {t4}\n')
print(f't5: {t5}\n')

### Numpy & Torch

In [ ]:
import numpy as np

##### **Creating a Numpy Array**

In [ ]:
n1 = np.array([[1, 2, 3], [3., 4, 5]])
print(n1)

Note that dtype is float due to the presence of a one "3.". This is because numpy array has uniformity too.

**Converting Numpy array to Torch Tensor**

In [ ]:
t6 = torch.from_numpy(n1)
print(t6)

**Converting Torch Tensor to Numpy Array**

In [ ]:
n2 = t6.numpy()
print(n2)

# Linear Regression
**Y = WX + B**
Y = Output, X = Input, W = Weights and B = Bias. Mathematically, **Y = X x W<sup>T</sup> + B**

Let us consider the following input dataset

| Rainfall (X1) | Temperature (X2) | Yield (Y) |
|      :-:      |      :-:         |      :-:  |
| 67 | 73 | 56 |
| 88 | 91 | 81 |
| 134 | 87 | 119 |
| 43 | 102 | 22 |
| 96 | 69 | 103 |

So, our Input Array X is

In [ ]:
X = np.array([[67,73],
              [88,91],
              [134,87],
              [43,102],
              [96,69]],dtype='float32')
print(X)

Our actual output vector Y is

In [ ]:
Y = np.array([56,81,119,22,103],dtype='float32')
print(Y)

Convert these to Torch Tensors

In [ ]:
X = torch.from_numpy(X)
Y = torch.from_numpy(Y)

Now, we need to obtain the weights W and biases B, of the best fit to this dataset. Initialize W and B with random values.

### Initialize Weights and Biases

In [ ]:
W = torch.randn(1,2,requires_grad=True)   # Because W Transpose should be 2x1
B = torch.randn(5,1,requires_grad=True)

### Define the Model

In [ ]:
def model(X):
    return X @ W.t() + B

**@** represents matrix multiplication in PyTorch and **.t()** returns Transpose of a tensor Let our predictions be **Y_**

In [ ]:
Y_ = model(X)

### Loss Function
Define the Loss Function. Considering **MSE** (Mean Squared Error) Loss

In [ ]:
def MSE(y,h):
    diff = h - y
    return torch.sum(diff**2)/diff.numel()

**torch.sum()** returns sum of elements of a tensor and .numel() returns the number of elements in a tensor.

**Compute the Loss for our model**

In [ ]:
Loss = MSE(Y,Y_)    

### Back Propogation

In [ ]:
Loss.backward()

### Compute Gradients

In [ ]:
W.grad     
B.grad

### Updation Step
Updating our weights W and biases B. Consider a learning rate of **a**. Let a = 1e-5

In [ ]:
a = 1e-5
with torch.no_grad():
    W -= a*(W.grad)
    B -= a*(B.grad) 

We use ````torch.no_grad()```` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

Now, reset the gradients to zero. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke backprop on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [ ]:
W.grad.zero_()
B.grad.zero_()

### Train for multiple Epochs
Repeat the steps until the loss is significantly minimized. For **N** Epochs

In [ ]:
N = 100
for i in range(N):
    Y_ = model(X)
    Loss = MSE(Y, Y_)
    Loss.backward()
    with torch.no_grad():
        W -= W.grad * a
        B -= B.grad * a
        W.grad.zero_()
        B.grad.zero_()

Compute the Loss now and verify that it is lower. Compare the actual outputs and predicted outputs.

In [ ]:
Loss = MSE(Y,Y_)
print(f'MSE Loss: {Loss}\n')
print(f'Predicted Output: {Y_}\n')
print(f'Actual Output: {Y}\n')

## Linear Regression using PyTorch packages
PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Import the ````torch.nn```` package, which contains utility classes for building neural networks.

In [38]:
import torch.nn as nn

Let us consider a bigger dataset, with 3 Input Parameters - Temperature, Humidity and Rainfall

Represent Inputs and Outputs as matrices

In [39]:
# Input (Temperature, Rainfall, Humidity)
X = np.array([[73, 67, 43], 
              [91, 88, 64], 
              [87, 134, 58], 
              [102, 43, 37], 
              [69, 96, 70], 
              [74, 66, 43], 
              [91, 87, 65], 
              [88, 134, 59], 
              [101, 44, 37], 
              [68, 96, 71], 
              [73, 66, 44], 
              [92, 87, 64], 
              [87, 135, 57], 
              [103, 43, 36], 
              [68, 97, 70]], 
              dtype='float32')

# Targets (Yield, say Mangoes and Bananas)
Y = np.array([[56, 70], 
              [81, 101], 
              [119, 133], 
              [22, 37], 
              [103, 119],
              [57, 69], 
              [80, 102], 
              [118, 132], 
              [21, 38], 
              [104, 118], 
              [57, 69], 
              [82, 100], 
              [118, 134], 
              [20, 38], 
              [102, 120]], 
              dtype='float32')

X = torch.from_numpy(X)
Y = torch.from_numpy(Y)

### Dataset and DataLoader
We'll create a ````TensorDataset````, which allows access to rows from inputs (X) and outputs (Y) as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [40]:
from torch.utils.data import TensorDataset

In [42]:
# Define dataset
train_ds = TensorDataset(X, Y)
print(train_ds[0:])

(tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]]), tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]]))


In [43]:
print(train_ds[0:5])

(tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]]), tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]]))


````TensorDataset```` allows us to access a small section of the training data using the array indexing notation (````[0:5]```` in the above example). It returns a tuple with two elements. The first element contains the ````inputs X```` for the selected rows, and the second contains the ````outputs Y````.

Import ````DataLoader````, which can split the data into batches of predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [44]:
from torch.utils.data import DataLoader

In [45]:
# Define Data Loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [48]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    print('\n')

tensor([[ 68.,  97.,  70.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 88., 134.,  59.],
        [ 91.,  87.,  65.]])
tensor([[102., 120.],
        [ 81., 101.],
        [ 22.,  37.],
        [118., 132.],
        [ 80., 102.]])


tensor([[101.,  44.,  37.],
        [103.,  43.,  36.],
        [ 74.,  66.,  43.],
        [ 73.,  67.,  43.],
        [ 68.,  96.,  71.]])
tensor([[ 21.,  38.],
        [ 20.,  38.],
        [ 57.,  69.],
        [ 56.,  70.],
        [104., 118.]])


tensor([[ 87., 134.,  58.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 69.,  96.,  70.],
        [ 87., 135.,  57.]])
tensor([[119., 133.],
        [ 57.,  69.],
        [ 82., 100.],
        [103., 119.],
        [118., 134.]])




In each iteration, DataLoader returns one batch of data with the given batch size. 

If ````shuffle=True````, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the ````nn.Linear```` class, which does it automatically.

In [49]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4375, -0.2529,  0.2407],
        [-0.2481, -0.3997, -0.1724]], requires_grad=True)
Parameter containing:
tensor([-0.5586,  0.0897], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. In our model, we have one weight matrix and one bias matrix.

In [50]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4375, -0.2529,  0.2407],
         [-0.2481, -0.3997, -0.1724]], requires_grad=True),
 Parameter containing:
 tensor([-0.5586,  0.0897], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [51]:
# Generate predictions
Y_ = model(X)
print(Y_)

tensor([[-39.0911, -52.2147],
        [-47.2228, -68.6943],
        [-58.5484, -85.0520],
        [-47.1542, -48.7843],
        [-38.1765, -67.4670],
        [-39.2757, -52.0631],
        [-46.7293, -68.4670],
        [-58.7452, -85.4726],
        [-46.9696, -48.9358],
        [-37.4983, -67.3912],
        [-38.5976, -51.9874],
        [-47.4075, -68.5428],
        [-59.0419, -85.2793],
        [-47.8324, -48.8601],
        [-37.9918, -67.6185]], grad_fn=<AddmmBackward>)


### Loss Function
Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`

In [52]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [54]:
# Define the Loss Function
loss_fn = F.mse_loss

Compute the Loss for the current predictions (Y_) of our model.

In [56]:
Loss = loss_fn(model(X), Y)
print(Loss)

tensor(21485.8887, grad_fn=<MseLossBackward>)


## Optimizer
Instead of manually manipulating the model weights & biases using gradients as done before, we can use the optimizer `optim.SGD` - SGD stands for "Stochastic Gradient Descent". SGD selects samples in random batches, instead as a single group.

In [57]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

`model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. We can also specify a learning rate `lr` that controls the amount by which the parameters are modified.

## Train the Model
Training the model constitutes:
* Generate predictions

* Calculate the Loss

* **Backpropogation**: Compute Gradients

* **Update Step**: Update weights and biases by subtracting `gradient*lr`

* Reset gradients to zero for the next iteration

Here we work with batches of data instead of processing the entire training data in every iteration. 

Define a function `LinReg` that trains our Linear Regression model for a given number of epochs.

In [58]:
# Function to Train the model
def LinReg(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for all epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # Generate predictions
            pred = model(xb)
            
            # Calculate the Loss
            Loss = loss_fn(pred, yb)
            
            # Backprop
            Loss.backward()
            
            # Update the parameters 
            opt.step()
            
            # Reset gradients to zero
            opt.zero_grad()
        
        # Print the Progress
        if (epoch+1) % 10 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {Loss.item()}')

* Use `DataLoader` to get batches of data for every iteration.

* We use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* Print the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Train the model for 100 epochs

In [59]:
LinReg(100, model, loss_fn, opt, train_dl)

Epoch: 10/100, Loss: 312.4222412109375
Epoch: 20/100, Loss: 340.64935302734375
Epoch: 30/100, Loss: 129.66961669921875
Epoch: 40/100, Loss: 171.03321838378906
Epoch: 50/100, Loss: 80.40342712402344
Epoch: 60/100, Loss: 41.6878547668457
Epoch: 70/100, Loss: 40.71399688720703
Epoch: 80/100, Loss: 38.065940856933594
Epoch: 90/100, Loss: 19.5218563079834
Epoch: 100/100, Loss: 18.048315048217773


Print out the predictions and actual outputs and compare them

In [60]:
print(f'Predictions: {model(X)}\n')
print(f'Actual Outputs: {Y}\n')

Predictions: tensor([[ 57.6125,  71.5877],
        [ 82.5405,  98.3715],
        [115.4930, 134.9106],
        [ 25.5902,  45.0049],
        [ 99.8847, 110.5835],
        [ 56.5155,  70.6304],
        [ 82.4459,  98.0328],
        [115.8567, 135.3188],
        [ 26.6872,  45.9623],
        [100.8871, 111.2021],
        [ 57.5178,  71.2490],
        [ 81.4435,  97.4141],
        [115.5876, 135.2493],
        [ 24.5879,  44.3863],
        [100.9817, 111.5408]], grad_fn=<AddmmBackward>)

Actual Outputs: tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])



For obtaining the prediction for a new input (Temperature, Rainfall and Humidity), we can pass a batch containing single row of input to our model.

In [61]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.5462, 68.4821]], grad_fn=<AddmmBackward>)